# Pattern: Get a Value While Ignoring One or More Exceptions

We often see situations like this one. Given a function `get_something()` which we have no permission to modify:

In [1]:
def do_something(n):
    if n < 10:
        raise ValueError(f"Invalid input: {n}")
    return n * 2

The pattern we see is this: call the function, if exception, return a different value:

In [2]:
value = 0
try:
    value = do_something(5)
except ValueError:
    pass
print(f"Value: {value}")

Value: 0


How about creating some function to deal with this?

In [4]:
import contextlib


def call_except(action, exceptions=Exception, default_return=None):
    with contextlib.suppress(exceptions):
        return action()
    return default_return

# Usage:
value = call_except(lambda: do_something(15))
print(f"For input 15, value={value}")

value = call_except(lambda: do_something(5), exceptions=ValueError, default_return="unable to calculate")
print(f"For input 5, value={value!r}")

For input 15, value=30
For input 5, value='unable to calculate'


Alternatively

In [5]:
def call_except2(function, *args, exceptions=Exception, default_return=None, **kwargs):
    with contextlib.suppress(exceptions):
        return function(*args, **kwargs)
    return default_return

# Usage:
value = call_except2(do_something, 15)
print(f"For input 15, value={value}")

value = call_except2(do_something, 5)
print(f"For input 5, value={value!r}")

For input 15, value=30
For input 5, value=None


## Ignore Using Context Manager


In [7]:
import contextlib

class IgnoreException:
    def __init__(self, exceptions, default=None):
        if not isinstance(exceptions, tuple):
            exceptions = (exceptions,)
        self.exceptions = exceptions
        self.default = default
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        pass
    
    def call(self, function, *args, **kwargs):
        with contextlib.suppress(self.exceptions):
            return function(*args, **kwargs)
        return self.default

In [8]:
with IgnoreException(ValueError, default="error found") as context:
    value = context.call(do_something, 5)
    print(f"value={value!r}")

value='error found'
